## Example NRS import

This notebook will import recordings in .nrs format from the cloud

In [ ]:
# Define some functions

import lbdb
import kachery as ka
from labbox_ephys import AutoRecordingExtractor, AutoSortingExtractor
import hither
import numpy as np

def geom_from_recording(recording):
    channel_ids = recording.get_channel_ids()
    location0 = recording.get_channel_property(channel_ids[0], 'location')
    nd = len(location0)
    M = len(channel_ids)
    geom = np.zeros((M, nd))
    for ii in range(len(channel_ids)):
        location_ii = recording.get_channel_property(channel_ids[ii], 'location')
        geom[ii, :] = list(location_ii)
    return geom

def import_recording(*, recording_id, recording_path):
    print(f'Importing {recording_id}')
    recording = AutoRecordingExtractor(recording_path, download=True)
    doc = dict(
        recording_id=recording_id,
        recording_path=recording_path,
        sampling_frequency=recording.get_sampling_frequency(),
        channel_ids=recording.get_channel_ids(),
        geom=geom_from_recording(recording).tolist(),
        num_frames=recording.get_num_frames()
    )
    lbdb.update(collection='recordings', query=dict(recording_id=recording_id), update=doc, upsert=True)

In [ ]:
# Import recordings from cloud

with ka.config(fr='default_readonly'):
    #
    import_recording(
        recording_id='nrs:128chan_sampleCA1',
        recording_path='sha1dir://80b58d66a95546365f2803993794f6cb7d7d31f8.128chan_sampleCA1'
    )

In [ ]:
# List all of the recordings in the database

recordings = lbdb.find(collection='recordings', query=dict())
for rec in recordings:
    print(rec['recording_id'])

In [ ]:
# List all of the sortings in the database

sortings = lbdb.find(collection='sortings', query=dict())
for rec in sortings:
    print(rec['sorting_id'])